In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
import optuna
from sklearn.model_selection import RandomizedSearchCV
import hyperopt as hp
import warnings
warnings.filterwarnings('ignore')

c:\Users\user\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\user\AppData\Local\Programs\Python\Python313\Lib\site-packages\hyperopt\atpe.py:19: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [2]:
data_cls = pd.read_csv('2dataset.csv')
data_cls = data_cls.sample(frac=0.1, random_state=42)
y_cls = data_cls['RainTomorrow']
X_cls = data_cls.drop(['RainTomorrow', 'Date', 'Location'], axis=1)

In [3]:
smote = SMOTE(random_state=42)
X_balanced, y_balanced = smote.fit_resample(X_cls, y_cls)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_balanced)

In [4]:
# Применение PCA (сохранение 95% дисперсии)
pca = PCA(n_components=0.95, random_state=42)
X_pca = pca.fit_transform(X_scaled)

print(f"Исходное количество признаков: {X_cls.shape[1]}, и после: {X_pca.shape[1]}")
print(f"Размер данных до SMOTE: {X_cls.shape[0]}, и после: {X_balanced.shape[0]}")

Исходное количество признаков: 23, и после: 16
Размер данных до SMOTE: 8698, и после: 14646


In [5]:
# K-fold кросс-валидация
k = 5
kf = KFold(n_splits=k, shuffle=True, random_state=42)
model = RandomForestClassifier(random_state=42)
scores = []

for train_index, test_index in kf.split(X_pca):
    X_train, X_test = X_pca[train_index], X_pca[test_index]
    y_train, y_test = y_balanced[train_index], y_balanced[test_index]

In [6]:
def objective_optuna(trial):
    n_layers = trial.suggest_int('n_layers', 1, 3)
    layers = []
    for i in range(n_layers):
        layers.append(trial.suggest_int(f'n_units_{i}', 10, 100))
    
    solver = trial.suggest_categorical('solver', ['adam', 'sgd', 'lbfgs'])
    params = {
        'hidden_layer_sizes': tuple(layers),
        'activation': trial.suggest_categorical('activation', ['relu', 'tanh']),
        'solver': solver,
        'learning_rate_init': trial.suggest_float('learning_rate_init', 1e-5, 1e-1, log=True) if solver != 'lbfgs' else 0.001,
        'max_iter': 100,
        'random_state': 42
    }
    
    model = MLPClassifier(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return accuracy_score(y_test, y_pred)

study = optuna.create_study(direction='maximize')
study.optimize(objective_optuna, n_trials=10)
optuna_params = study.best_params
optuna_score = study.best_value

print("\nOptuna Results:")
print(f"Best parameters: {optuna_params}")
print(f"Best accuracy: {optuna_score:.4f}")

[I 2025-06-02 15:38:47,733] A new study created in memory with name: no-name-5456f42b-aa1c-4ea7-af0a-e937835010d2
[I 2025-06-02 15:38:48,842] Trial 0 finished with value: 0.7791054967565723 and parameters: {'n_layers': 1, 'n_units_0': 42, 'solver': 'sgd', 'activation': 'tanh', 'learning_rate_init': 0.0021120975048242886}. Best is trial 0 with value: 0.7791054967565723.
[I 2025-06-02 15:38:50,142] Trial 1 finished with value: 0.8289518607033117 and parameters: {'n_layers': 1, 'n_units_0': 56, 'solver': 'adam', 'activation': 'tanh', 'learning_rate_init': 0.0020835185156305473}. Best is trial 1 with value: 0.8289518607033117.
[I 2025-06-02 15:38:51,007] Trial 2 finished with value: 0.8047115056333219 and parameters: {'n_layers': 1, 'n_units_0': 26, 'solver': 'sgd', 'activation': 'tanh', 'learning_rate_init': 0.06357046128963402}. Best is trial 1 with value: 0.8289518607033117.
[I 2025-06-02 15:38:54,654] Trial 3 finished with value: 0.8552406964834415 and parameters: {'n_layers': 3, 'n_un


Optuna Results:
Best parameters: {'n_layers': 3, 'n_units_0': 42, 'n_units_1': 57, 'n_units_2': 11, 'solver': 'adam', 'activation': 'relu', 'learning_rate_init': 0.015193994997706607}
Best accuracy: 0.8614


In [9]:
# 2. RandomizedSearchCV
param_dist = {
    'hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 50), (50, 25)],
    'activation': ['relu', 'tanh'],
    'solver': ['adam', 'sgd', 'lbfgs'],
    'learning_rate_init': np.logspace(-5, -1, 100),
    'max_iter': [100]
}

model = MLPClassifier(random_state=42)
random_search = RandomizedSearchCV(model, param_distributions=param_dist, 
                                 n_iter=10, cv=3, random_state=42)
random_search.fit(X_train, y_train)
random_params = random_search.best_params_
random_score = random_search.best_score_

print("\nRandomizedSearchCV Results:")
print(f"Best parameters: {random_params}")
print(f"Best accuracy: {random_score:.4f}")



RandomizedSearchCV Results:
Best parameters: {'solver': 'adam', 'max_iter': 100, 'learning_rate_init': np.float64(0.0042292428743894986), 'hidden_layer_sizes': (100, 50), 'activation': 'relu'}
Best accuracy: 0.8682


In [ ]:
# 3. Hyperopt
def objective_hyperopt(params):
    learning_rate = params['learning_rate_init'] if params['solver'] != 'lbfgs' else 0.001
    model = MLPClassifier(
        hidden_layer_sizes=params['hidden_layer_sizes'],
        activation=params['activation'],
        solver=params['solver'],
        learning_rate_init=learning_rate,
        max_iter=100,
        random_state=42
    )
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return -accuracy_score(y_test, y_pred)

space = {
    'hidden_layer_sizes': hp.hp.choice('hidden_layer_sizes', 
                                  [(50,), (100,), (50, 50), (100, 50), (50, 25)]),
    'activation': hp.hp.choice('activation', ['relu', 'tanh']),
    'solver': hp.hp.choice('solver', ['adam', 'sgd', 'lbfgs']),
    'learning_rate_init': hp.hp.loguniform('learning_rate_init', -5, -1)
}

trials = hp.Trials()
hyperopt_params = hp.fmin(fn=objective_hyperopt,
                      space=space,
                      algo=hp.tpe.suggest,
                      max_evals=10,
                      trials=trials)
hyperopt_score = -min(trials.losses())

print("\nHyperopt Results:")
print(f"Best parameters: {hyperopt_params}")
print(f"Best accuracy: {hyperopt_score:.4f}")

100%|██████████| 20/20 [03:06<00:00,  9.35s/trial, best loss: -0.8163950143815916]

Hyperopt Results:
Best parameters: {'activation': np.int64(0), 'hidden_layer_sizes': np.int64(3), 'learning_rate_init': np.float64(0.018206606977761358)}
Best accuracy: 0.8164


In [ ]:
import numpy as np

# Базовый класс для слоев
class Layer:
    def forward(self, x):
        raise NotImplementedError
    
    def backward(self, grad_output):
        raise NotImplementedError

# Аффинный слой (WX + b)
class DenseLayer(Layer):
    def __init__(self, input_size, output_size):
        self.W = np.random.randn(input_size, output_size) * 0.01
        self.b = np.zeros((1, output_size))
        self.input = None
        
    def forward(self, x):
        self.input = x
        return np.dot(x, self.W) + self.b
    
    def backward(self, grad_output):
        grad_input = np.dot(grad_output, self.W.T)
        grad_W = np.dot(self.input.T, grad_output)
        grad_b = np.sum(grad_output, axis=0, keepdims=True)
        return grad_input, {'W': grad_W, 'b': grad_b}

# Классы функций активации
class ReLU(Layer):
    def __init__(self):
        self.input = None
        
    def forward(self, x):
        self.input = x
        return np.maximum(0, x)
    
    def backward(self, grad_output):
        grad_input = grad_output * (self.input > 0)
        return grad_input, {}

class Tanh(Layer):
    def __init__(self):
        self.input = None
        
    def forward(self, x):
        self.input = x
        return np.tanh(x)
    
    def backward(self, grad_output):
        grad_input = grad_output * (1 - np.tanh(self.input) ** 2)
        return grad_input, {}

class Sigmoid(Layer):
    def __init__(self):
        self.input = None
        
    def forward(self, x):
        self.input = x
        return 1 / (1 + np.exp(-x))
    
    def backward(self, grad_output):
        sigmoid = 1 / (1 + np.exp(-self.input))
        grad_input = grad_output * sigmoid * (1 - sigmoid)
        return grad_input, {}

# Классы функций потерь
class Loss:
    def compute(self, y_pred, y_true):
        raise NotImplementedError
    
    def gradient(self, y_pred, y_true):
        raise NotImplementedError

class MSELoss(Loss):
    def compute(self, y_pred, y_true):
        return np.mean((y_pred - y_true) ** 2)
    
    def gradient(self, y_pred, y_true):
        return 2 * (y_pred - y_true) / y_pred.shape[0]

class CrossEntropyLoss(Loss):
    def compute(self, y_pred, y_true):
        y_pred = np.clip(y_pred, 1e-15, 1 - 1e-15)
        return -np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))
    
    def gradient(self, y_pred, y_true):
        y_pred = np.clip(y_pred, 1e-15, 1 - 1e-15)
        return (y_pred - y_true) / (y_pred * (1 - y_pred) * y_pred.shape[0])

# Класс оптимизатора
class SGDOptimizer:
    def __init__(self, learning_rate=0.01):
        self.lr = learning_rate
    
    def update(self, layer, gradients):
        if hasattr(layer, 'W') and 'W' in gradients:
            layer.W -= self.lr * gradients['W']
            layer.b -= self.lr * gradients['b']

# Класс нейронной сети
class NeuralNetwork:
    def __init__(self, layers, loss_fn, optimizer):
        self.layers = layers
        self.loss_fn = loss_fn
        self.optimizer = optimizer
    
    def forward(self, x):
        for layer in self.layers:
            x = layer.forward(x)
        return x
    
    def backward(self, grad_output):
        gradients = []
        for layer in reversed(self.layers):
            grad_input, grad_params = layer.backward(grad_output)
            gradients.append(grad_params)
            grad_output = grad_input
        return list(reversed(gradients))
    
    def train(self, x, y, epochs=100):
        for epoch in range(epochs):
            # Прямое распространение
            y_pred = self.forward(x)
            
            # Вычисление потерь
            loss = self.loss_fn.compute(y_pred, y)
            
            # Обратное распространение
            grad_output = self.loss_fn.gradient(y_pred, y)
            gradients = self.backward(grad_output)
            
            # Обновление весов
            for layer, grad in zip(self.layers, gradients):
                self.optimizer.update(layer, grad)
                
            if epoch % 10 == 0:
                print(f"Epoch {epoch}, Loss: {loss:.4f}")
    
    def predict(self, x):
        return self.forward(x)